# Sacar los datos

http://www.mineduc.gob.gt/BUSCAESTABLECIMIENTO_GE/ 


In [4]:
import sys
!{sys.executable} -m pip install --user "selenium"

  Using cached typing_extensions-4.14.1-py3-none-any.whl.metadata (3.0 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 6.1 MB/s eta 0:00:00a 0:00:01m
Using cached typing_extensions-4.14.1-py3-none-any.whl (43 kB)

[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip3 install --upgrade pip


In [5]:
import zipfile
import urllib.request
import io
import pandas as pd
import xml.etree.ElementTree as ET
import json

import requests
from bs4 import BeautifulSoup #pip install bs4 por si debe instalarlo
import camelot #pip install "camelot-py[cv]" por si debe instalarlo

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
import time, os

In [3]:
url =  "http://www.mineduc.gob.gt/BUSCAESTABLECIMIENTO_GE/ "
response = requests.request("GET", url)
web_data = BeautifulSoup(response.text,'html.parser')
print(web_data)

<!DOCTYPE html>

<html>
<head>
<title>The resource cannot be found.</title>
<meta content="width=device-width" name="viewport"/>
<style>
         body {font-family:"Verdana";font-weight:normal;font-size: .7em;color:black;} 
         p {font-family:"Verdana";font-weight:normal;color:black;margin-top: -5px}
         b {font-family:"Verdana";font-weight:bold;color:black;margin-top: -5px}
         H1 { font-family:"Verdana";font-weight:normal;font-size:18pt;color:red }
         H2 { font-family:"Verdana";font-weight:normal;font-size:14pt;color:maroon }
         pre {font-family:"Consolas","Lucida Console",Monospace;font-size:11pt;margin:0;padding:0.5em;line-height:14pt}
         .marker {font-weight: bold; color: black;text-decoration: none;}
         .version {color: gray;}
         .error {margin-bottom: 10px;}
         .expandable { text-decoration:underline; font-weight:bold; color:navy; cursor:pointer; }
         @media screen and (max-width: 639px) {
          pre { width: 440px; ove

In [7]:

# Configuración
driver = webdriver.Chrome()  # Asegúrate de tener chromedriver en PATH
wait = WebDriverWait(driver, 15)
driver.get("https://www.mineduc.gob.gt/BUSCAESTABLECIMIENTO_GE/")

output_dir = "excels_centros"
os.makedirs(output_dir, exist_ok=True)

# 1. Obtener la lista de departamentos
select_depto = Select(wait.until(EC.element_to_be_clickable((By.ID, "ctl00_ContentPlaceHolder1_ddlDepartamento"))))
departamentos = [opt.get_attribute("value") for opt in select_depto.options if opt.get_attribute("value") != ""]

for depto_value in departamentos:
    select_depto.select_by_value(depto_value)
    # 2. Click en buscar
    btn_buscar = driver.find_element(By.ID, "ctl00_ContentPlaceHolder1_btnBuscar")
    btn_buscar.click()
    # 3. Esperar a que aparezca la tabla
    wait.until(EC.visibility_of_element_located((By.ID, "ctl00_ContentPlaceHolder1_gvResultado")))

    # Cuántas filas hay
    rows = driver.find_elements(By.CSS_SELECTOR, "#ctl00_ContentPlaceHolder1_gvResultado tr")[1:]
    print(f"Departamento {depto_value}, {len(rows)} centros encontrados.")

    for i, row in enumerate(rows, start=1):
        # Botón “Seleccionar centro”
        btn_sel = row.find_element(By.LINK_TEXT, "Seleccionar centro")
        btn_sel.click()

        # Esperar y click al botón de descarga en la ventana emergente/modal
        # Asumiendo que aparece nuevo botón con id "ctl00_ContentPlaceHolder1_btnDescargar"
        download_btn = wait.until(EC.element_to_be_clickable((By.ID, "ctl00_ContentPlaceHolder1_btnDescargar")))
        # Obtener nombre de archivo desde el centro (ejemplo: texto de cabecera del modal)
        nombre_centro = wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, ".modal-title"))).text
        safe_name = "".join(c for c in nombre_centro if c.isalnum() or c in " -_").rstrip()
        path = os.path.join(output_dir, f"{safe_name}.xlsx")

        # Redirigir descarga automática
        download_btn.click()
        # Esperar que el archivo aparezca localmente (puede necesitar configuración del navegador)
        time.sleep(5)
        print(f"Descargado: {safe_name}.xlsx")

        # Cerrar modal o regresar a la tabla
        btn_cerrar = driver.find_element(By.CSS_SELECTOR, ".modal-close")  # ajusta selector
        btn_cerrar.click()
        wait.until(EC.visibility_of_element_located((By.ID, "ctl00_ContentPlaceHolder1_gvResultado")))

    # Para la siguiente iteración, volver a recargar la lista de departamentos
    select_depto = Select(driver.find_element(By.ID, "ctl00_ContentPlaceHolder1_ddlDepartamento"))
    driver.get(driver.current_url)  # refrescar si hay problema

driver.quit()

TimeoutException: Message: 
Stacktrace:
0   chromedriver                        0x0000000100ef4e6c cxxbridge1$str$ptr + 2722840
1   chromedriver                        0x0000000100eecd74 cxxbridge1$str$ptr + 2689824
2   chromedriver                        0x0000000100a3e3ec cxxbridge1$string$len + 90648
3   chromedriver                        0x0000000100a85544 cxxbridge1$string$len + 381808
4   chromedriver                        0x0000000100ac6934 cxxbridge1$string$len + 649056
5   chromedriver                        0x0000000100a79834 cxxbridge1$string$len + 333408
6   chromedriver                        0x0000000100eb7f88 cxxbridge1$str$ptr + 2473268
7   chromedriver                        0x0000000100ebb1f4 cxxbridge1$str$ptr + 2486176
8   chromedriver                        0x0000000100e999d0 cxxbridge1$str$ptr + 2348924
9   chromedriver                        0x0000000100ebbab0 cxxbridge1$str$ptr + 2488412
10  chromedriver                        0x0000000100e8aa60 cxxbridge1$str

In [8]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC

driver = webdriver.Chrome()
driver.get("https://apps2.mineduc.gob.gt/UTE/ConsultaEstablecimientos.aspx")
wait = WebDriverWait(driver, 20)

# Esperar que aparezca el select
try:
    select_elem = wait.until(EC.presence_of_element_located((By.ID, "ctl00_ContentPlaceHolder1_ddlDepartamento")))
    select = Select(select_elem)
    for opt in select.options:
        print(opt.text)
except:
    print("No se encontró el menú de departamentos.")

No se encontró el menú de departamentos.


In [6]:
from selenium.webdriver.chrome.options import Options
opt = Options()
opt.add_experimental_option("prefs", {
    "download.default_directory": os.path.abspath(output_dir),
    "download.prompt_for_download": False,
    "plugins.always_open_pdf_externally": True
})
driver = webdriver.Chrome(options=opt)

NameError: name 'output_dir' is not defined